# Step 1 - Calculations

#import packpages

In [ ]:
import math as m
import cmath
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
plt.rcParams['text.usetex'] = True

import scipy.interpolate as interp

# 1.1 - Standard Model Parameters

Mixing angles and CPT phase

In [ ]:
pi = m.pi
theta12 = (pi * 34.3) / 180
theta13 = (pi * 8.53) / 180
theta23 = (pi * 49.26) / 180
cptphase = (pi * 194) / 180

Squared mass difference

In [ ]:
Deltam21 = 7.50e-5
Deltam31 = 2.55e-3
Deltam32 = Deltam21+Deltam31

Experiment baseline [km]

In [ ]:
baseline = 1300

Flavour oscilattion (va -> vb) (0 = e, 1 = muon, 2 = tau)

In [ ]:
a = 1
b = 1

Mixing matrix calculation

In [ ]:
mixing_matrix_SM =  np.array([[np.cos(theta12) * np.cos(theta13),    np.sin(theta12)*np.cos(theta13),    np.sin(theta13)*cmath.exp(-cptphase * 1j)],
                     [-np.sin(theta12)*np.cos(theta23)-np.cos(theta12)*np.sin(theta23)*np.sin(theta13)*cmath.exp(cptphase * 1j),     np.cos(theta12)*np.cos(theta23)-np.sin(theta12)*np.sin(theta23)*np.sin(theta13)*cmath.exp(cptphase * 1j),   np.sin(theta23)*np.cos(theta13)],
                     [np.sin(theta12)*np.cos(theta23)-np.cos(theta12)*np.cos(theta23)*np.sin(theta13)*cmath.exp(cptphase * 1j),     -np.cos(theta12)*np.sin(theta23)-np.sin(theta12)*np.cos(theta23)*np.sin(theta13)*cmath.exp(cptphase * 1j),   np.cos(theta23)*np.cos(theta13)]], dtype = complex)
                     

mass_matrix_SM =  np.array([[0,     Deltam21,   Deltam31],
                            [Deltam21,     0 ,  Deltam32],
                            [Deltam31,  Deltam32,      0]])

In [ ]:
#kronecker delta
if a == b:
    kdelta = 1
else:
    kdelta = 0

# 1.2 - Puma Model Parameters


In [ ]:
#unit correction
factor = 5.07e18
#ap = -2.5e-19
#cp = 1.0e-16
m2 = 2.6e-23

# 1.3 Chi Squared calculation

In [ ]:
#creating a list
ap_cp_chi = []

#for ap in np.arange(-11.00,-6.01, 0.05):
for ap in np.arange(-5.96,-5.88, 0.001):  
    
    #for cp in np.arange(7.50, 28.01, 0.05):
    for cp in np.arange(5.30, 5.44, 0.001):
        Chi_Sum = 0

        #E = Energy range [GeV]     
        for E in np.arange(0.5, 10, 0.5):
            #Standard Model
            #sum
            sum_real_SM = 0
            sum_img_SM = 0

            for j in range(0,3):
                for i in range(0,3):
                    if j>i:
                        sum_real_SM = sum_real_SM + np.conjugate(mixing_matrix_SM[a][i].real) * mixing_matrix_SM[b][i].real * mixing_matrix_SM[a][j].real * np.conjugate(mixing_matrix_SM[b][j].real) * (np.sin(1.27 * baseline * mass_matrix_SM[j][i]/E))**2
                        sum_img_SM = sum_img_SM + np.conjugate(mixing_matrix_SM[a][i].imag) * mixing_matrix_SM[b][i].imag * mixing_matrix_SM[a][j].imag * np.conjugate(mixing_matrix_SM[b][j].imag) * np.sin(1.27 * baseline * mass_matrix_SM[j][i]/E)

            #Probility oscilattion (va -> vb)
            #(± sign in the third term is the CP violation effect, − for neutrinos and + for anti-neutrinos)
            Pab_SM_MM = kdelta - 4 * sum_real_SM - 2 * sum_img_SM
           
           
            #PUMA MODEL
            #A = m2/ (2 * E)
            B = ap * 1e-19 * E**2
            C = cp * 1e-16 * E**5 + 1.2e-22
            
            #Probility oscilattion (v_muon -> v_muon)
            Pvmtovm_Puma = 1 - (np.sin((baseline * B**2 * factor)/(C)))**2
            
            #print('Puma: ' + str(Pvmtovm_Puma) + ' SM: ' + str(Pab_SM_MM) + ' dif: ' + str(Pab_SM_MM - Pvmtovm_Puma) + ' χ^2_element: ' + str((Pab_SM_MM - Pvmtovm_Puma)**2 / (Pab_SM_MM * 0.1)**2))
            
            Chi_Sum = Chi_Sum + ((Pab_SM_MM - Pvmtovm_Puma)**2 / (Pab_SM_MM * 0.1)**2)
            #print('Value of a = ' + str(ap) + ' Value of c = ' + str(cp) +  ', Energy(GeV) = ' + str(E))
        
        # print('Value of a = ' + str(ap) + ' Value of c = ' + str(cp) + ' Chi_Squared = ' + str(Chi_Sum))
        # print('') 
    
        ap_cp_chi.append((ap,cp,Chi_Sum))
    print('Value of a = ' + str(ap))

Creating the Dataframe

In [ ]:
df_ap_cp_chi = pd.DataFrame(ap_cp_chi, columns=['a','c', 'Chi_Squared_for_a_c'])

Adding Delta Chi diference

In [ ]:
delta_chi_ap_cp = []

for value in df_ap_cp_chi['Chi_Squared_for_a_c']:
    delta_chi_ap_cp.append(value - df_ap_cp_chi['Chi_Squared_for_a_c'].min())
    
df_ap_cp_chi["Delta_Chi"] = delta_chi_ap_cp

Viewing the data

In [ ]:
print(df_ap_cp_chi)

# 2 -  Graphics Analisys

# 2.1 - Find the Minimum χ 

Creating filters for maximum Chi values (If you don't want to filter any values add # to this block of code)

In [ ]:
filter_ap_cp = df_ap_cp_chi['Delta_Chi'] <= df_ap_cp_chi['Delta_Chi'].min() + 20
df_ap_cp_chi = df_ap_cp_chi[filter_ap_cp].reset_index(drop=True)
print(df_ap_cp_chi)

Find the minimum chi value and index in the data.

In [ ]:
min_df_ap_cp_chi = df_ap_cp_chi['Delta_Chi'].min()
ind_min_df_ap_cp_chi = df_ap_cp_chi['Chi_Squared_for_a_c'].idxmin()
df_ap_cp_chi_min = df_ap_cp_chi.iloc[ind_min_df_ap_cp_chi]

print("Index of Chi Squared minimum: " + str(ind_min_df_ap_cp_chi))
print("Value of Chi Squared minimum, a and c")
print(df_ap_cp_chi_min)

Excluding χ columns (Working with Δχ)

In [ ]:
df_ap_cp_chi = df_ap_cp_chi.loc[:, df_ap_cp_chi.columns!='Chi_Squared_for_a_c']
print(df_ap_cp_chi)

# 2.2 - Confindence Level Analysis

In [ ]:
#creating the filters
one_sigma_ap_cp = df_ap_cp_chi['Delta_Chi'] <= min_df_ap_cp_chi + 2.30
two_sigma_ap_cp = df_ap_cp_chi['Delta_Chi'] <= min_df_ap_cp_chi + 4.61
three_sigma_ap_cp = df_ap_cp_chi['Delta_Chi'] <= min_df_ap_cp_chi + 5.99
four_sigma_ap_cp = df_ap_cp_chi['Delta_Chi'] <= min_df_ap_cp_chi + 6.18
five_sigma_ap_cp = df_ap_cp_chi['Delta_Chi'] <= min_df_ap_cp_chi + 9.21
six_sigma_ap_cp = df_ap_cp_chi['Delta_Chi'] <= min_df_ap_cp_chi + 11.83

#aplying the filters and creating a dataframe with all values
df_1_sigma_ap_cp = df_ap_cp_chi[one_sigma_ap_cp]
df_2_sigma_ap_cp = df_ap_cp_chi[two_sigma_ap_cp]
df_3_sigma_ap_cp = df_ap_cp_chi[three_sigma_ap_cp]
df_4_sigma_ap_cp = df_ap_cp_chi[four_sigma_ap_cp]
df_5_sigma_ap_cp = df_ap_cp_chi[five_sigma_ap_cp]
df_6_sigma_ap_cp = df_ap_cp_chi[six_sigma_ap_cp]

Plot confindence level ("a" and "c" on axis)

In [ ]:
#confidence level regions
ax = df_6_sigma_ap_cp.plot(kind='scatter', x='a', y='c', color='y', label= r"$6\sigma - 99.93 \% $")
df_5_sigma_ap_cp.plot(kind='scatter', x='a', y='c', color='m', ax = ax, label= r"$5\sigma - 99.00 \% $")
df_4_sigma_ap_cp.plot(kind='scatter', x='a', y='c', color='c', ax = ax, label= r"$4\sigma - 95.45 \% $")
df_3_sigma_ap_cp.plot(kind='scatter', x='a', y='c', color='r', ax = ax, label= r"$3\sigma - 95.00 \% $")
df_2_sigma_ap_cp.plot(kind='scatter', x='a', y='c', color='g', ax = ax, label= r"$2\sigma - 90.00 \% $")
df_1_sigma_ap_cp.plot(kind='scatter', x='a', y='c', color='k', ax = ax, label= r"$1\sigma - 68.27 \% $")


#Labels of the axis and title
ax.set_xlabel(r'a $= (10^{-16})$  [GeV]\textsuperscript{-4}')
ax.set_ylabel(r'c $= (10^{-19})$  [GeV]\textsuperscript{-1}')
ax.set_title('Confindence Level Region')

#Place a legend on the axes.
ax.legend()

plt.show()

# 2.3 - 3D surface

In [ ]:
%matplotlib notebook
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(projection='3d')

# Plot the surface
surf = ax.plot_trisurf(df_1_sigma_ap_cp['a'], df_1_sigma_ap_cp['c'], df_1_sigma_ap_cp['Delta_Chi'], color='k', label= r"$1\sigma - 68.27 \% $")
surf._facecolors2d = surf._facecolor3d 
surf._edgecolors2d = surf._edgecolor3d

surf = ax.plot_trisurf(df_2_sigma_ap_cp['a'], df_2_sigma_ap_cp['c'], df_2_sigma_ap_cp['Delta_Chi'], color='g', label= r"$2\sigma - 90.00 \% $")
surf._facecolors2d = surf._facecolor3d 
surf._edgecolors2d = surf._edgecolor3d

surf = ax.plot_trisurf(df_3_sigma_ap_cp['a'], df_3_sigma_ap_cp['c'], df_3_sigma_ap_cp['Delta_Chi'], color='r', label= r"$3\sigma - 95.00 \% $")
surf._facecolors2d = surf._facecolor3d 
surf._edgecolors2d = surf._edgecolor3d

surf = ax.plot_trisurf(df_4_sigma_ap_cp['a'], df_4_sigma_ap_cp['c'], df_4_sigma_ap_cp['Delta_Chi'], color='c', label= r"$4\sigma - 95.45 \% $")
surf._facecolors2d = surf._facecolor3d 
surf._edgecolors2d = surf._edgecolor3d


surf = ax.plot_trisurf(df_5_sigma_ap_cp['a'], df_5_sigma_ap_cp['c'], df_5_sigma_ap_cp['Delta_Chi'], color='m', label= r"$5\sigma - 99.00 \% $")
surf._facecolors2d = surf._facecolor3d 
surf._edgecolors2d = surf._edgecolor3d

surf = ax.plot_trisurf(df_6_sigma_ap_cp['a'], df_6_sigma_ap_cp['c'], df_6_sigma_ap_cp['Delta_Chi'], color='y', label= r"$6\sigma - 99.93 \% $")
surf._facecolors2d = surf._facecolor3d 
surf._edgecolors2d = surf._edgecolor3d

surf = ax.plot_trisurf(df_ap_cp_chi['a'], df_ap_cp_chi['c'], df_ap_cp_chi['Delta_Chi'], color='b')
surf._facecolors2d = surf._facecolor3d 
surf._edgecolors2d = surf._edgecolor3d

#Labels of the axis and title
ax.set_xlabel(r'a $= (10^{-16})$  [GeV]\textsuperscript{-4}')
ax.set_ylabel(r'c $= (10^{-19})$  [GeV]\textsuperscript{-1}')
ax.set_zlabel(r'$\Delta \chi^2$')
ax.set_title(r'$\Delta \chi^2 \times a \times c$')
ax._facecolors2d = ax._facecolor

#Place a legend on the axes.
ax.legend()

#Scale of the axis
# ax.set_xscale("linear", "log")
# ax.set_yscale("linear", "log")

#Ticks location
# ax.xaxis.set_major_locator(plt.MultipleLocator(0.02e-19))
# ax.xaxis.set_minor_locator(plt.MultipleLocator(0.01e-19))
# ax.yaxis.set_major_locator(plt.MultipleLocator(10))
# ax.yaxis.set_minor_locator(plt.MultipleLocator(2))

#Axis delimitation region 
# ax.set_xlim(left= , right= )
# ax.set_ylim(left= , right= )

plt.show()

In [ ]:
#convert dataframe to list
df_ap_cp_chi_plot = df_ap_cp_chi.transpose().values.tolist()
df_1_sigma_ap_cp_plot = df_1_sigma_ap_cp.transpose().values.tolist()
df_2_sigma_ap_cp_plot = df_2_sigma_ap_cp.transpose().values.tolist()
df_3_sigma_ap_cp_plot = df_3_sigma_ap_cp.transpose().values.tolist()
df_4_sigma_ap_cp_plot = df_4_sigma_ap_cp.transpose().values.tolist()
df_5_sigma_ap_cp_plot = df_5_sigma_ap_cp.transpose().values.tolist()
df_6_sigma_ap_cp_plot = df_6_sigma_ap_cp.transpose().values.tolist()

# Creating figure
fig = plt.figure(figsize=(7, 7))
ax = plt.axes(projection ='3d')
 
# Creating plot

surf = ax.plot_trisurf(df_ap_cp_chi_plot[0], df_ap_cp_chi_plot[1], df_ap_cp_chi_plot[2],zorder=0, linewidth = 0.2, color='b');
surf._facecolors2d = surf._facecolor3d 
surf._edgecolors2d = surf._edgecolor3d

surf = ax.plot_trisurf(df_6_sigma_ap_cp_plot[0], df_6_sigma_ap_cp_plot[1], df_6_sigma_ap_cp_plot[2], linewidth = 0.2, antialiased = True, color='y', label= r"$6\sigma - 99.93 \% $")
surf._facecolors2d = surf._facecolor3d 
surf._edgecolors2d = surf._edgecolor3d

surf = ax.plot_trisurf(df_5_sigma_ap_cp_plot[0], df_5_sigma_ap_cp_plot[1], df_5_sigma_ap_cp_plot[2], linewidth = 0.2, antialiased = True, color='m', label= r"$5\sigma - 99.00 \% $")
surf._facecolors2d = surf._facecolor3d 
surf._edgecolors2d = surf._edgecolor3d

surf = ax.plot_trisurf(df_4_sigma_ap_cp_plot[0], df_4_sigma_ap_cp_plot[1], df_4_sigma_ap_cp_plot[2], linewidth = 0.2, antialiased = True, color='c', label= r"$4\sigma - 95.45 \% $")
surf._facecolors2d = surf._facecolor3d 
surf._edgecolors2d = surf._edgecolor3d

surf = ax.plot_trisurf(df_3_sigma_ap_cp_plot[0], df_3_sigma_ap_cp_plot[1], df_3_sigma_ap_cp_plot[2], linewidth = 0.2, antialiased = True, color='r', label= r"$3\sigma - 95.00 \% $")
surf._facecolors2d = surf._facecolor3d 
surf._edgecolors2d = surf._edgecolor3d

surf = ax.plot_trisurf(df_2_sigma_ap_cp_plot[0], df_2_sigma_ap_cp_plot[1], df_2_sigma_ap_cp_plot[2], linewidth = 0.2, antialiased = True, color='g', label= r"$2\sigma - 90.00 \% $")
surf._facecolors2d = surf._facecolor3d 
surf._edgecolors2d = surf._edgecolor3d

surf = ax.plot_trisurf(df_1_sigma_ap_cp_plot[0], df_1_sigma_ap_cp_plot[1], df_1_sigma_ap_cp_plot[2], linewidth = 0.2, antialiased = True, color='k', label= r"$1\sigma - 68.27 \% $")
surf._facecolors2d = surf._facecolor3d 
surf._edgecolors2d = surf._edgecolor3d



#Labels of the axis and title
ax.set_xlabel(r'a$(10^{-16})$  [GeV]\textsuperscript{-4}')
ax.set_ylabel(r'c$(10^{-19})$  [GeV]\textsuperscript{-1}')
ax.set_zlabel(r'$\Delta \chi^2$')
ax.set_title(r'$\Delta \chi^2 \times a \times c$')


#Place a legend on the axes.
ax.legend()

#Scale of the axis
# ax.set_xscale("linear", "log")
# ax.set_yscale("linear", "log")

#Ticks location
# ax.xaxis.set_major_locator(plt.MultipleLocator(0.02e-19))
# ax.xaxis.set_minor_locator(plt.MultipleLocator(0.01e-19))
# ax.yaxis.set_major_locator(plt.MultipleLocator(10))
# ax.yaxis.set_minor_locator(plt.MultipleLocator(2))

#Axis delimitation region 
# ax.set_xlim(left= , right= )
# ax.set_ylim(left= , right= )
#ax.set_zlim(0, 40)

plt.show()

    
# show plot
plt.show()


Projections of "a" for the fixed value of "c" associated with the χ^2 minimum

In [ ]:
#creating a dataframe
df_ap_cp_min_chi = df_ap_cp_chi[df_ap_cp_chi['c'] == df_ap_cp_chi_min['c']]

#ploting
fig, ax = plt.subplots()
df_ap_cp_min_chi.plot(kind='line',x='a', y='Delta_Chi',color='k', ax=ax)

#Labels of the axis and title
ax.set_xlabel(r'a $= (10^{-16})$  [GeV]\textsuperscript{-4}')
ax.set_ylabel(r'$\Delta \chi^2$')
ax.set_title(r'$\Delta \chi^2 \times a$')


plt.show()

Projections of "c" for the fixed value of "" associated with the χ^2 minimum

In [ ]:
#creating a dataframe
df_ap_min_cp_chi = df_ap_cp_chi[df_ap_cp_chi['a'] == df_ap_cp_chi_min['a']]

#ploting
fig, ax = plt.subplots()
df_ap_min_cp_chi.plot(kind='line',x='c', y='Delta_Chi',color='k' , ax=ax)

#Labels of the axis and title
ax.set_xlabel(r'c $= (10^{-19})$  [GeV]\textsuperscript{-1}')
ax.set_ylabel(r'$\Delta \chi^2$')
ax.set_title(r'$\Delta \chi^2 \times c$')

plt.show()